<a href="https://colab.research.google.com/github/venkatab144/UTS_ML2019_ASS2_12954525/blob/master/ML_A2_12954525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The CART (Classification And Regression Trees) algorithm, classifies data by splitting attributes based on their gini index. CART starts with the root node, and generates a split by asking a question which results in the highest information gain. This is done by calculating the Gini index of the dataset from the root node, and then subtracting the weighted average of the Gini index of the resulting datasets from the split.

The Gini index of a dataset is calculated by calculating the probability that a randomly chosen label within a given amount of rows is correct. This can be used to measure the impurity of data. A Gini index of 1 means that there is a 0% chance of incorrectly assigning a label and that the data is pure. A low Gini index means that there is a higher risk of assigning an incorrect label to a row and this results in a higher Gini impurity. Therefore, it can be said that impurity is inversely proportional to the Gini index.

The inputs for this CART implementation is an array of rows which contain attributes and a label in the final column. The ouput will be the classes that were predicted from the tree and confidence (in %) of the prediction.

# Exploration

**Relevant Terminology:**

**Gini Index:**
- Gini index measures the impurity of a dataset by calculating the probablilty that a row of data belongs to a certain label/class.

**Root Node:**
- The starting node which contains the entire dataset and is split into child nodes.

**Parent Node:**
- A node which contains child nodes.

**Child Node:**
- A node which is created when a parent node is split.

**Splitting:**
- The process of dividing the dataset into subsets based off a question, and results in new sub-trees.

**Question:**
- Gives a true/false answer based on a certain condition which is based on an attribute.

**Sub-tree:**
- A new tree created from splitting a node.

**Leaf Node:**
- A terminal node at the end of a tree/sub-tree. It contains the predicted labels for a class.

The structure for the CART algorithm is as follows:
1. The Gini index of the entire dataset for the root node is calculated using the following formula:
<center>GINI FORMULA</center>
2. A "question" is proposed based off each attribute and the question with the highest information gain is selected.
3. True and False nodes are created for this split.
4. Step 1 is recursivley called on these nodes until a <i>leaf</i> node is reached.
> A leaf node is the terminal node for a specific branch of a tree. It contains the predicted labels for the row when that branch is followed. A leaf node only exists when the information gain is 0 and there are no further questions to ask.
5. Once all leaf nodes have been created and no more useful (questions that have some information gain), have been asked, the model has completed training and is ready to classify data.

Planned Approach/Structure:
1. Starting from the root node, go through every attribute and generate a question.
2. To generate a question, we first check if the value is numeric or not.
3. If the value is numeric, we generate a question to check if all other numeric attributes are greater than or equal to its value
4. If the value is non-numeric, we check if all other attributes are an exact match.
5. We calculate the Gini impurity of the current node.
6. We then split the data based on the question and generate true and false rows.
7. We calculate the weighted average of the Gini impurity of these true/false nodes.
8. We the calculate the information gain by subtracting the Gini impurity of the true/false child nodes from the parent node, and store the question with the highest information gain.
9. Once we have the best possible split, if the information gain is 0, that means no new information can be gained and a leaf node is created.
10. If there is information to be gained, we recursivley create new trees from the true and false branches.


# Methodology

## !!!!! This section will only list and explain each method. A full, working and testable implementation will be located at the bottom of this notebook in the "*Full Implementation*" section. !!!!!

## Main Methods

To start the process, the algorithm needs to take the entire training dataset and start constructing the full tree, starting with the root node. It then needs to find the best query that splits the node resulting in the most information gain and then creates sub-trees from the split node. This can be seen in the `create_tree()` method below:

In [0]:
def create_tree(self, data_rows, is_root_node=False):
        # We need to find the split that results in the lowest Gini index and
        # the highest information gain by proposing a question from the given data_rows
        
        # Create a new node object to store the current node
        current_node = self.Node(data_rows=data_rows, is_root_node=is_root_node)
        
        # Calculate the split which produces the best information gain and store
        # the query, so that we know what needs to be asked where
        best_gain, best_query = self.find_best_split(current_node)
        
        # If the split results in 0 gain, this means that there is no more information
        # to be gained from this node and we can store it as a Terminal node.
        if (best_gain == 0):
            current_node.populate_node(best_gain, best_query, None, None)
            
            # Get the labels that are left in the split dataset and store them 
            # as the predictions
            predicted_labels, predicted_counts = self.get_label_counts(current_node)
            current_node.predictions = predicted_labels
        else:
            # If there IS information to be gained, we need to store the true
            # and false nodes and create new sub-trees from them
            true_rows, false_rows = self.create_split(data_rows, best_query)
            
            true_node = self.create_tree(true_rows.data_rows)
            false_node = self.create_tree(false_rows.data_rows)
            
            # Populate the node with details for visualization
            current_node.populate_node(best_gain, best_query, true_node, false_node)
        
        # Once we've finished our recursion and end up back at the root node
        # we can store the root node in a variable to access it later for prediction
        if (current_node.is_root_node):
            print("Finished")
            self.trained_model = current_node
        
        return current_node

Next we need to be able to find what the best query is for the highest information gain. This is achieved by iterating through all attributes and their values, creating a query and splitting the data based on if the other values match it. We then calculate the Gini impurity using the following formula: $$Gini(X) = 1-\sum_{i=1}^C (p_{i})^2$$ where $p_{i}$ is the prbability that row $X$ belongs to class $C$. 

In [0]:
def calculate_gini_impurity(self, node):
    # Get the labels and their counts for calculation and index purposes
    labels, counts = self.get_label_counts(node)
    
    # We need to know the total number of rows to calculate the probablity that
    # a class belongs to a row
    num_rows = node.num_rows()
    impurity = 1
    
    # Calculate the Gini impurity of that node.
    for label in labels:
        label_probabililty = counts[label] / float(num_rows)
        impurity -= label_probabililty**2

    return impurity

In [0]:
def find_best_split(self, parent_node):
        # We need to store the best gain and the best query so we can correctly path
        # values from predictions
        best_info_gain = 0
        best_query = None
        
        # We need the parent node gini impurity to be able to calculate the information
        # gain
        parent_gini = self.calculate_gini_impurity(parent_node)
        
        # We need to now start constructing queries for every single attribute and
        # their values in order to find the best query to split the node with
        for column in parent_node.attribute_columns():
            # Gets all the unique values for that column
            for value in parent_node.data_rows[column].unique():
            
                query = self.Query(column, value)
                
                # Create a split and generate the true and false nodes so that we
                # can evaluate the information gain
                true_node, false_node = self.create_split(parent_node.data_rows, query)
                
                # If the split generates an empty true/false column we skip it as 
                # its redundant and does not help us
                if (true_node.num_rows() > 0 and false_node.num_rows() > 0):
                    # We need to know the info gain of this split so we can store
                    # the best split
                    info_gain = self.calculate_info_gain(parent_gini, true_node, false_node);\
    
                    # Store the best gain
                    if (info_gain >= best_info_gain):
                        best_info_gain = info_gain
                        best_query = query
                    
        return best_info_gain, best_query    

We need to also calculate the information gain by subtracting the weighted average of true and false nodes from the parent Gini impurity, and that is done through: $$g_{total} = g_{parent} - (p_{true} * g_{true}) - (p_{false} * g_{false})$$ where $g$ is gain and $p$ is how often it appears in the dataset.

In [0]:
def calculate_info_gain(self, parent_gini, true_node, false_node):
    # Calculate the Gini impurity of the true node and false node
    gini_true = self.calculate_gini_impurity(true_node)
    gini_false = self.calculate_gini_impurity(false_node)
    
    # Calculate the weights of each node
    true_weight = float(true_node.num_rows()) / (true_node.num_rows() + false_node.num_rows())
    false_weight = 1 - true_weight
    
    # Calculate the total information gain
    info_gain = parent_gini - ((true_weight * gini_true) + (false_weight * gini_false))
    
    return info_gain

A split needs to be create which evaluates the query for each row and creates the corresponding true/false nodes:

In [0]:
def create_split(self, data_rows, query):
    # New nodes for true/false branches
    true_node, false_node = self.Node(pd.DataFrame()), self.Node(pd.DataFrame())

    # If a row matches the query, append it to the true node and if it doesnt,
    # append it to the false node
    for index, row in data_rows.iterrows():
        if (query.evaluate_result(row)):
            true_node.append_row(row)
        else:
            false_node.append_row(row)
        
    return true_node, false_node

## Fit, Classify and Predict methods

In [0]:
def fit(self, training_data):
    # Initialize the the root node and recursion process
    return self.create_tree(training_data, True)
    
def classify(self, row, tree):
    # If we have reached a prediction then display the results
    if (tree.is_terminal_node()):
        print("Actual:", row[-1], "Predicted:", tree.predictions[0])
        return tree.predictions
    
    # Otherwise match the row with the query and follow the true/false branch
    if (tree.query.evaluate_result(row)):
        return self.classify(row, tree.true_node)
    else:
        return self.classify(row, tree.false_node)

def predict(self, data_rows):
    # Prevent prediction from an incomplete model
    if (self.trained_model is None):
        print("Please train the model with fit(training_data) first.")
        return
    
    # Otherwise, classify each row.
    for row in data_rows.iterrows():
        self.classify(row[1], self.trained_model)

## Query and Node Sub-classes

In [0]:
class Node:
    def __init__(self, data_rows, is_root_node=False):
        self.data_rows = data_rows
        self.query = None
        self.info_gain = 0
        self.true_node = None
        self.false_node = None
        self.node_type = ""
        self.is_root_node = is_root_node
        self.predictions = None
        
    def populate_node(self, gain, query, true_node, false_node):
        self.info_gain = gain
        self.query = query
        self.true_node = true_node
        self.false_node = false_node
        
        if (gain == 0):
            self.node_type = "Terminal"
        else:
            self.node_type = "Split"
        
    def num_rows(self):
        return self.data_rows.shape[0]
    
    def attribute_columns(self):
        return self.data_rows.columns[:-1]
    
    def label_column(self):
        return self.data_rows.columns[-1]
    
    def append_row(self, row):
        self.data_rows = self.data_rows.append(row)
        
    def is_terminal_node(self):
        return self.node_type == "Terminal"
    
    def is_split_node(self):
        return self.node_type == "Split"

    class Query:
        def __init__(self, attribute, value):
            self.attribute = attribute
            self.value = value
        
        def is_number(self, object_to_check):
            return isinstance(object_to_check, Number)
            
        def evaluate_result(self, row):
            value_to_evaluate = row[self.attribute]
            
            if(self.is_number(value_to_evaluate)):
                return value_to_evaluate >= self.value
            else:
                return value_to_evaluate == self.value
        
        def query_desc(self):
            desc = self.attribute
            if(self.is_number(self.value)):
                desc += " >= "
            else:
                desc += " == "
            desc += str(self.value)
            
            return desc

## Helper Classes

In [0]:
class Logger:
    def __init__(self):
        self.logs = []
    
    def log(self, *args):
        self.logs.append(' '.join(map(str, args)))
        
    def print_logs(self):
        for log in self.logs:
            print(log)

## Usage
### !!! Please check "*Full Implementation*" section for a working solution. !!!

In [0]:
classifier = DecisionTreeClassifier()

classifier.fit(training_data)
classifier.predict(testing_data)

## Output

```
Finished
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: versicolor Predicted: versicolor
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa
Actual: virginica Predicted: virginica
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: virginica Predicted: virginica
Actual: virginica Predicted: virginica
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa 
```

# Evaluation

When tested on the Iris dataset using a training/test split of 80:20, the implementation presented in this report achieves a 100% accuracy rate. However, the model that is generated by the above implementation will be heavily over-fitted to whatever dataset it is trained with, and any new data will require the model to be re-trained in order to maintain its accuracy.

A potential solution to this is utilizing k-fold cross validation in order to generate multiple folds of the training set, allowing the model to be trained and tested on a 'k' number of different training/testing sets. This will help reduce the presence over-fitting and will reduce the impact that new data will have on an already trained model.

The algorithm presented is also not optimised to achieve the best performance and is not the most efficient. It is a basic implementation of the CART algorithm and as such, does not include methods such as pruning and max-depth configuration. This specific implementation of the CART algorithm utilizes the pandas library and makes use of DataFrames heavliy which are more computationally expensive than using simple lists. It is also quite object-oriented; nodes and queries are stored in their own objects and occupy unnecessary memory. Objects that end up having no use are created in abundance (e.g. in find_best_split() true/false nodes are created for every possible attribute/value combination, as well as questions), without being properly destroyed, resulting in heavy memory usage. Proper garbage collection was not able to be achieved in the time frame imposed on this assignment. 

Decision Tree (DT) algorithms such as CART are white box learning algorithms used for classification and regression. The internal logic used by the DT can be viewed unlike other black box algorithms. Compared to other classification algorithms such as: KNN and SVM, the process followed by the is viewable and it is often possible to construct flowcharts of all the possible node paths leading to which prediction was made. There are many flaws of decision tree algorithms such as CART, small changes can have massive impacts on the model, larger datasets make it extremely computationally expensive and at times it can make flawed decision and assumptions causing incorrect predictions.

# Conclusion

In conclusion, through the implementation above, it can be seen that the CART algorithm can be successfully used to classify data accurately. However, limitations have also been outlined and there are many possible areas for improvement, such as: the use of k-fold cross validation to reduce over-fitting,using simple lists instead of DataFrames, pruning and max-dept customization, proper garbage collection and a reduced reliance on object-orientation.

Implementing these improvements could very likely increase the efficiency of the algorithm and also increase its accuracy. Given more time, this algorithm could be optimised to perform faster, use less memory and increase its accuracy. This implementation would still not be suitable to larger datasets. If a problem exists that can be solved with decision tree classifiers it would be more worthy to use existing libraries such as scikit-learn, but, if full control and customization is required, like in an enterprise environment, it would be worthwhile to implement the algorithm from scratch and build to to suit the required needs.

# Ethical

The CART algorithm is a Supervised Learning machine learning task. There are many potential areas for misuse in the Machine Learning field. Recently, the concept of Deepfakes have surfaced and it has been shown that Deepfakes can successfully be used to superimpose an individuals face onto anothers body. Deepfakes employ the Generative Adversarial Networks (GAN) machine learning technique in order to be used in a malicous manner. The GAN technique utilizes supervized learning methods such as classification and regression, which is what the CART algorithm specializes in (Classification and Regression Trees).

The Utilitarian approach to ethics weighs the value of an action based on the outcome is achieves. The CART algorithm used for unlawful actions would produce an unethical and unlawful outcome that would hurt the individuals that are affected by it. 

The Deontological Kantian Based approach argues that rather than analysing the affects and repercussions of an action to weigh whether it was right or wrong, that one should instead query the motives of the individual or entity that performed the action. Applied in this context, if the CART algorithm is used in a malicious way, the ethical problem lies not with the original author but with the person/enitity that used the algorithm for their unethical motives.

# Video Pitch

Highlights Challenges and Effort

# Full Implementation

### Please Note: This code is not optimised and can take upto a minute to complete.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from numbers import Number

iris = sns.load_dataset('iris')

training_data, testing_data = train_test_split(iris, test_size=0.2)

class Logger:
    def __init__(self):
        self.logs = []
    
    def log(self, *args):
        self.logs.append(' '.join(map(str, args)))
        
    def print_logs(self):
        for log in self.logs:
            print(log)

class DecisionTreeClassifier:
    def __init__(self):
        self.trained_model= None

    def get_label_counts(self, node):
        counts = node.data_rows.iloc[:,-1].value_counts();
        labels = counts.index;
        return labels, counts
    
    class Node:
        def __init__(self, data_rows, is_root_node=False):
            self.data_rows = data_rows
            self.query = None
            self.info_gain = 0
            self.true_node = None
            self.false_node = None
            self.node_type = ""
            self.is_root_node = is_root_node
            self.predictions = None
            
        def populate_node(self, gain, query, true_node, false_node):
            self.info_gain = gain
            self.query = query
            self.true_node = true_node
            self.false_node = false_node
            
            if (gain == 0):
                self.node_type = "Terminal"
            else:
                self.node_type = "Split"
            
        def num_rows(self):
            return self.data_rows.shape[0]
        
        def attribute_columns(self):
            return self.data_rows.columns[:-1]
        
        def label_column(self):
            return self.data_rows.columns[-1]
        
        def append_row(self, row):
            self.data_rows = self.data_rows.append(row)
            
        def is_terminal_node(self):
            return self.node_type == "Terminal"
        
        def is_split_node(self):
            return self.node_type == "Split"
            
    
    def calculate_gini_impurity(self, node):
        # Get the labels and their counts for calculation and index purposes
        labels, counts = self.get_label_counts(node)
        
        # We need to know the total number of rows to calculate the probablity that
        # a class belongs to a row
        num_rows = node.num_rows()
        impurity = 1
        
        # Calculate the Gini impurity of that node.
        for label in labels:
            label_probabililty = counts[label] / float(num_rows)
            impurity -= label_probabililty**2
    
        return impurity
    
    class Query:
        def __init__(self, attribute, value):
            self.attribute = attribute
            self.value = value
        
        def is_number(self, object_to_check):
            return isinstance(object_to_check, Number)
            
        def evaluate_result(self, row):
            value_to_evaluate = row[self.attribute]
            
            if(self.is_number(value_to_evaluate)):
                return value_to_evaluate >= self.value
            else:
                return value_to_evaluate == self.value
        
        def query_desc(self):
            desc = self.attribute
            if(self.is_number(self.value)):
                desc += " >= "
            else:
                desc += " == "
            desc += str(self.value)
            
            return desc
            
            
    def create_split(self, data_rows, query):
        # New nodes for true/false branches
        true_node, false_node = self.Node(pd.DataFrame()), self.Node(pd.DataFrame())

        # If a row matches the query, append it to the true node and if it doesnt,
        # append it to the false node
        for index, row in data_rows.iterrows():
            if (query.evaluate_result(row)):
                true_node.append_row(row)
            else:
                false_node.append_row(row)
         
        return true_node, false_node
    
    def calculate_info_gain(self, parent_gini, true_node, false_node):
        # Calculate the Gini impurity of the true node and false node
        gini_true = self.calculate_gini_impurity(true_node)
        gini_false = self.calculate_gini_impurity(false_node)
        
        # Calculate the weights of each node
        true_weight = float(true_node.num_rows()) / (true_node.num_rows() + false_node.num_rows())
        false_weight = 1 - true_weight
        
        # Calculate the total information gain
        info_gain = parent_gini - ((true_weight * gini_true) + (false_weight * gini_false))
        
        return info_gain
    
    def find_best_split(self, parent_node):
        # We need to store the best gain and the best query so we can correctly path
        # values from predictions
        best_info_gain = 0
        best_query = None
        
        # We need the parent node gini impurity to be able to calculate the information
        # gain
        parent_gini = self.calculate_gini_impurity(parent_node)
        
        # We need to now start constructing queries for every single attribute and
        # their values in order to find the best query to split the node with
        for column in parent_node.attribute_columns():
            # Gets all the unique values for that column
            for value in parent_node.data_rows[column].unique():
            
                query = self.Query(column, value)
                
                # Create a split and generate the true and false nodes so that we
                # can evaluate the information gain
                true_node, false_node = self.create_split(parent_node.data_rows, query)
                
                # If the split generates an empty true/false column we skip it as 
                # its redundant and does not help us
                if (true_node.num_rows() > 0 and false_node.num_rows() > 0):
                    # We need to know the info gain of this split so we can store
                    # the best split
                    info_gain = self.calculate_info_gain(parent_gini, true_node, false_node);\
    
                    # Store the best gain
                    if (info_gain >= best_info_gain):
                        best_info_gain = info_gain
                        best_query = query
                    
        return best_info_gain, best_query    
        
    
    def create_tree(self, data_rows, is_root_node=False):
        # We need to find the split that results in the lowest Gini index and
        # the highest information gain by proposing a question from the given data_rows
        
        # Create a new node object to store the current node
        current_node = self.Node(data_rows=data_rows, is_root_node=is_root_node)
        
        # Calculate the split which produces the best information gain and store
        # the query, so that we know what needs to be asked where
        best_gain, best_query = self.find_best_split(current_node)
        
        # If the split results in 0 gain, this means that there is no more information
        # to be gained from this node and we can store it as a Terminal node.
        if (best_gain == 0):
            current_node.populate_node(best_gain, best_query, None, None)
            
            # Get the labels that are left in the split dataset and store them 
            # as the predictions
            predicted_labels, predicted_counts = self.get_label_counts(current_node)
            current_node.predictions = predicted_labels
        else:
            # If there IS information to be gained, we need to store the true
            # and false nodes and create new sub-trees from them
            true_rows, false_rows = self.create_split(data_rows, best_query)
            
            true_node = self.create_tree(true_rows.data_rows)
            false_node = self.create_tree(false_rows.data_rows)
            
            # Populate the node with details for visualization
            current_node.populate_node(best_gain, best_query, true_node, false_node)
        
        # Once we've finished our recursion and end up back at the root node
        # we can store the root node in a variable to access it later for prediction
        if (current_node.is_root_node):
            print("Finished")
            self.trained_model = current_node
        
        return current_node
    
    def fit(self, training_data):
        # Initialize the the root node and recursion process
        return self.create_tree(training_data, True)
        
    def classify(self, row, tree):
        # If we have reached a prediction then display the results
        if (tree.is_terminal_node()):
            print("Actual:", row[-1], "Predicted:", tree.predictions[0])
            return tree.predictions
        
        # Otherwise match the row with the query and follow the true/false branch
        if (tree.query.evaluate_result(row)):
            return self.classify(row, tree.true_node)
        else:
            return self.classify(row, tree.false_node)
    
    def predict(self, data_rows):
        # Prevent prediction from an incomplete model
        if (self.trained_model is None):
            print("Please train the model with fit(training_data) first.")
            return
        
        # Otherwise, classify each row.
        for row in data_rows.iterrows():
            self.classify(row[1], self.trained_model)

classifier = DecisionTreeClassifier()

classifier.fit(training_data)
classifier.predict(testing_data)


Finished
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: versicolor Predicted: versicolor
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa
Actual: virginica Predicted: virginica
Actual: versicolor Predicted: versicolor
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: virginica Predicted: virginica
Actual: virginica Predicted: virginica
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: setosa Predicted: setosa
Actual: virginica Predicted: virginica
Actual: setosa Predicted: setosa
Actual: versicolor Predicted: versicolor
Actual: versicolor Predicted: versicol